In [6]:
import os
import numpy as np
from plot_params import *
from scipy.spatial.distance import cdist
from copy import deepcopy
from tqdm import tqdm
from joblib import Parallel, delayed

In [7]:
def total_energy(pos):
    dist = cdist(pos, pos)
    li = np.tril_indices(len(pos), k=-1)
    pair_dist = dist[li]
    total_energy = (1 / pair_dist).sum()
    return total_energy

def accept_move(T, energy_after, energy_before):
    delta_energy = energy_after - energy_before
    if delta_energy < 0:
        return True
    p = np.exp(- delta_energy / T)
    if np.random.rand() <= p:
        return True
    return False

def random_move(pos, i):
    og = deepcopy(pos)
    pos[i] = og[i] + np.random.uniform(-1, 1, 2) * stepsize
    if np.sqrt(pos[i][0] ** 2 + pos[i][1] ** 2) < circle_radius:
        return pos
    return og

def run():
    # initialize particles
    r = np.random.rand(n)
    theta = np.random.rand(n) * 2 * np.pi
    x = np.cos(theta) * r
    y = np.sin(theta) * r
    pos = np.stack((x, y), axis=1)

    # loop temp
    for j in tqdm(range(steps)):
        # loop charge
        for i in range(n):
            energy_before = total_energy(pos)
            possible_pos = random_move(deepcopy(pos), i)
            energy_after = total_energy(possible_pos)
            if accept_move(T[j], energy_after, energy_before):
                pos = possible_pos
                
    return pos

In [8]:
# settings
steps = 250
stepsize = 0.05
n = 3
runs = 50
circle_radius = 1
cpu_count = os.cpu_count()

# temp function
i = np.linspace(10, 1, steps)
B = 7
vu = 4
M = 6
T = 1 / (1 + np.exp(- B * (i - M)) ** (1 / vu)) # logistic because cool

# run 
results = Parallel(n_jobs=cpu_count, verbose=1)(delayed(run)() for _ in range(runs))

# save
results = np.array(results)
np.save(f'{runs}times{n}.npy', results)


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done  50 out of  50 | elapsed:    0.2s finished


In [9]:
# plt.figure(figsize=(6, 6))
# plt.scatter(pos[:, 0], pos[:, 1])
# circle = plt.Circle((0, 0), 1, fill=False)
# plt.gca().add_patch(circle)
# plt.show()

In [10]:
# i = np.linspace(10, 1, steps)
# B = 7
# vu = 4
# M = 6
# T = 1 / (1 + np.exp(- B * (i - M)) ** (1 / vu))

# plt.plot(i, T)
